In [ ]:
from utils.generator import *

In [ ]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
def add_transparent_image(background, foreground, x_offset=None, y_offset=None):
    bg_h, bg_w, bg_channels = background.shape
    fg_h, fg_w, fg_channels = foreground.shape

    assert bg_channels == 3, f'background image should have exactly 3 channels (RGB). found:{bg_channels}'
    assert fg_channels == 4, f'foreground image should have exactly 4 channels (RGBA). found:{fg_channels}'

    # center by default
    if x_offset is None: x_offset = (bg_w - fg_w) // 2
    if y_offset is None: y_offset = (bg_h - fg_h) // 2

    w = min(fg_w, bg_w, fg_w + x_offset, bg_w - x_offset)
    h = min(fg_h, bg_h, fg_h + y_offset, bg_h - y_offset)

    if w < 1 or h < 1: return

    # clip foreground and background images to the overlapping regions
    bg_x = max(0, x_offset)
    bg_y = max(0, y_offset)
    fg_x = max(0, x_offset * -1)
    fg_y = max(0, y_offset * -1)
    foreground = foreground[fg_y:fg_y + h, fg_x:fg_x + w]
    background_subsection = background[bg_y:bg_y + h, bg_x:bg_x + w]

    # separate alpha and color channels from the foreground image
    foreground_colors = foreground[:, :, :3]
    alpha_channel = foreground[:, :, 3] / 255  # 0-255 => 0.0-1.0

    # construct an alpha_mask that matches the image shape
    alpha_mask = np.dstack((alpha_channel, alpha_channel, alpha_channel))

    # combine the background with the overlay image weighted by alpha
    composite = background_subsection * (1 - alpha_mask) + foreground_colors * alpha_mask

    # overwrite the section of the background image that has been updated
    background[bg_y:bg_y + h, bg_x:bg_x + w] = composite

    return background

In [ ]:
def make_dataset_frame(frame, bg, transparency=220, color=np.float32([220,50,50]), kernel=7):
    frame = np.transpose(frame, axes=[1, 0, 2])
    frame = np.flip(frame, axis=0)
    bg = cv2.resize(bg, (frame.shape[1], frame.shape[0]))

    frame *= 255
    frame[:,:,3] = np.mean(frame[:,:,:3], axis=2)
    frame[np.where(frame[:,:,3] > 0)] = transparency
    frame[:,:,:3] = color
    frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, np.ones((kernel,kernel)))

    add_transparent_image(bg, frame)

    return bg

def make_dataset_video(video, **kwargs):
    files = os.listdir('backgrounds/')
    bg = None 
    while bg is None:
        bg = cv2.imread(f'backgrounds/{random.choice(files)}')

    new_video = np.zeros((*video.shape[:3], 3), dtype=np.uint8)
    p_color = np.uint8([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)])
    transparency = random.randint(127, 255)

    for i in range(video.shape[0]):
        new_video[i] = make_dataset_frame(
            video[i].copy(), 
            bg.copy(), 
            transparency=transparency, 
            color=p_color,
              **kwargs
        )

    return new_video

In [ ]:
from plot2vid import PlotRecorder

def get_sample(material, name, time_in_seconds=4):
    center = [0.1 + 0.8 * random.random(), 0.1 + 0.8 * random.random()]
    max_radius = min(center[0], center[1], 1 - center[0], 1 - center[1])
    radius = min(0.1 + 0.5 * random.random(), max_radius)

    p, v = generate_granular_trajectory(
        material, 
        time_in_seconds=time_in_seconds, 
        video_resolution=64, 
        sim_resolution=12, 
        center=center,
        radius=radius
    )
    vid = make_dataset_video(v, kernel=5)

    recorder = PlotRecorder(f'{name}.mp4', fps=15)
    fig, axs = plt.subplots(1, 2, figsize=(8,4), dpi=100)

    for i in tqdm(range(vid.shape[0])):
        if i % 5 == 0:
            axs[0].imshow(vid[i])
            axs[1].clear()
            axs[1].scatter(p[i,:,0], p[i,:,1])
            axs[1].axis('equal')
            axs[1].set_xlim(0,1.0)
            axs[1].set_ylim(0,1)

            recorder.add(fig)

    recorder.close()

    return p, vid

In [ ]:
parts, frames = get_sample('WATER', f'datasets/fluids/videos/water_test_{traj_number}')

In [ ]:
SAMPLES_PER_CLASS = 30

In [ ]:
for traj_number in range(SAMPLES_PER_CLASS):
    parts, frames = get_sample('WATER', f'datasets/fluids/videos/water_test_{traj_number}')
    torch.save(parts, f'datasets/fluids/particles/water_test_{traj_number}')
    torch.save(frames, f'datasets/fluids/frames/water_test_{traj_number}')

In [ ]:
for traj_number in range(SAMPLES_PER_CLASS):
    parts, frames = get_sample('SAND', f'datasets/fluids/videos/sand_test_{traj_number}')
    torch.save(parts, f'datasets/fluids/particles/sand_test_{traj_number}')
    torch.save(frames, f'datasets/fluids/frames/sand_test_{traj_number}')

In [ ]:
for traj_number in range(SAMPLES_PER_CLASS):
    parts, frames = get_sample('SNOW', f'datasets/fluids/videos/snow_test_{traj_number}')
    torch.save(parts, f'datasets/fluids/particles/snow_test_{traj_number}')
    torch.save(frames, f'datasets/fluids/frames/snow_test_{traj_number}')

In [ ]:
for traj_number in range(SAMPLES_PER_CLASS):
    parts, frames = get_sample('ELASTIC', f'datasets/fluids/videos/elastic_test_{traj_number}')
    torch.save(parts, f'datasets/fluids/particles/elastic_test_{traj_number}')
    torch.save(frames, f'datasets/fluids/frames/elastic_test_{traj_number}')